# Abstract Instruments and paramters

Abstract parameters allow us to create abstract instrument types which are guarenteed to have certain parameters present. For instance, this will allow us to create a unified interface for all voltage sources.

Usually, when a parameter is added to an instrument and that instrument is subclassed and a parameter of the same name is added in the subclass as well, this will raise an exception. This issue is circumvented by:

1. creating an abstract instrument by using the `abstract_instrument` class decorator, which serves as a base class. 
2. specifying in the base class that the parameter being added there is of type AbstractParameter.

A catch is that, if a unit is specified in the base class and a unit is specified in the sub class, these units must match.

In [1]:
from qcodes import Instrument 
from qcodes.instrument.abstract import AbstractParameter, abstract_instrument

In [2]:
@abstract_instrument
class BaseVoltageSource(Instrument): 
    """
    We demand that all voltage sources implement a standard interface
    defined by this base class 
    """
    
    def __init__(self, name: str): 
        super().__init__(name)
        
        self.add_parameter(
            "voltage", 
            unit="V", 
            parameter_class=AbstractParameter
        )
        
        self.add_parameter(
            "current", 
            unit="A", 
            get_cmd=None, 
            set_cmd=None 
        )

In [3]:
class MyVoltageSource(BaseVoltageSource): 
    """
    A voltage source driver for a particular instrument 
    make and model. 
    """
    
    def __init__(self, name: str): 
        super().__init__(name)
        
        self.add_parameter(
            "voltage", 
            unit="V", 
            set_cmd=None, 
            get_cmd=None
        )

In [4]:
# We can use this voltage source like any other QCoDeS driver 
source = MyVoltageSource("name1")
source.voltage(0.1)
print(source.voltage())

0.1


In [5]:
# Paramters other then 'AbstractParameter' are inherited in the usual way 
source.current(-0.1)
print(source.current())

-0.1


In [5]:
class WrongSource(BaseVoltageSource): 
    """
    Let's 'forget' to implement the voltage parameter
    """

In [6]:
WrongSource("name2")  # This will go badly wrong because we have not implemented an abstract parameter :-)

AbstractParameterException: Class 'WrongSource' un-implemented Abstract Parameter(s): 'voltage'

In [7]:
class WrongSource2(BaseVoltageSource): 
    """
    We implement the voltage paramter with the wrong unit 
    """
    
    def __init__(self, name: str): 
        super().__init__(name)
        
        self.add_parameter(
           "voltage", 
           unit="mV"
        )   

In [8]:
WrongSource2("name4")  # An excpetion will be raised, complaining about unit mismatch. 

AbstractParameterException: The unit of the parameter 'voltage' is 'mV', which is inconsistent with the unit 'V' specified earlier. This is usually because a driver is a subclass of a baseclass which defines a parameter of the same name but with different units

In [15]:
class A: 
    def f(self): 
        print("this is A")

class B: 
    def f(self): 
        super().f()
        print("this is B")

class C(A, B): 
    def f(self): 
        super().f()
        print("this is C")

In [16]:
c = C()

In [17]:
c.f()

this is A
this is C
